# Dependencies

In [1]:
import collections

import datasets
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchtext

/home/rvn/.envs/torch-2.2.0-cu-12.1/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Seeding to ensure reproducibility
seed = 0

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# IMDB Dataset

Loading the dataset is very easy using the `datasets` package! The command below will download (if required in the *data/classification* directory) and load the IMDB dataset in a variable named `dataset`.

In [3]:
dataset = datasets.load_dataset("zapsdcn/imdb", cache_dir="../data/classification/")

The loaded `dataset` is already split into training, validation and testing sets. Looking at the outputs, we can see the number of examples (`num_rows`) in each split and three features named `id`, `text` and `label`.

In [4]:
train_dataset = dataset['train'] 
validation_dataset = dataset['validation']
test_dataset = dataset['test']

train_dataset, validation_dataset, test_dataset

(Dataset({
     features: ['id', 'text', 'label'],
     num_rows: 20000
 }),
 Dataset({
     features: ['id', 'text', 'label'],
     num_rows: 5000
 }),
 Dataset({
     features: ['id', 'text', 'label'],
     num_rows: 25000
 }))

Looking at one such training example, we can see that 
- `id` contains information related to some index. Don't bother too much about this as this is not relevant for the application at hand.
- `text` is where we have the actual review!
- `label` denotes whether it's a good (if 1) or bad (if 0) review! 

In [5]:
train_dataset[-3]

{'id': 'train_10348',
 'text': 'This movie is very good. The screenplay is enchanting. But Meryl Streep is most impressive. Her performance is excellent. She brings me to go into the heart of her role.',
 'label': 1}

## Tokenization and Vocabulary Building

Computers only understand numbers. So we have to find a way to modify such that we can represent each word as a number. But first we have to split sentences into individual words. This is called Tokenization and is done using a tokenizer! 

I'm using a simple tokenizer provided by `torchtext`.

In [6]:
tokenizer = torchtext.data.utils.get_tokenizer("basic_english")

As expected, a tokenizer splits a text into words!

In [ ]:
tokenizer(train_dataset[-3]["text"])

Let's do this for all of the training, validation and testing dataset.

In [7]:
def tokenize(example, tokenizer, max_length):
    tokens = tokenizer(example["text"])[:max_length]
    return {"tokens": tokens}

In [8]:
# Truncating the max length of a review because some of them ramble on for quite some time!
max_length = 512

train_dataset = train_dataset.map(
    tokenize, fn_kwargs={"tokenizer": tokenizer, "max_length": max_length}
)
validation_dataset = validation_dataset.map(
    tokenize, fn_kwargs={"tokenizer": tokenizer, "max_length": max_length}
)
test_dataset = test_dataset.map(
    tokenize, fn_kwargs={"tokenizer": tokenizer, "max_length": max_length}
)

Let's build a volcabulary out of these tokens.

In [9]:
# Only selecting words at occur at least 5 times in the whole dataset
min_freq = 5

# Special tokens for unknown words and paddings (these will come in later)
special_tokens = ["<unk>", "<pad>"]

vocab = torchtext.vocab.build_vocab_from_iterator(
    train_dataset["tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [ ]:
len(vocab), vocab.get_itos()

Making sure that words that are not in the vocabulary gets mapped to `"<unk>"` and assiging a token for padding as well (we'll cover padding later).

In [10]:
unk_index = vocab["<unk>"]
pad_index = vocab["<pad>"]

vocab.set_default_index(unk_index)

vocab["some_token_that_is_not_in_vocab"]

0

Now with this we can easily map any review to a sequence of numbers!

In [ ]:
vocab.lookup_indices(train_dataset[-3]["tokens"])

Let's do this in a more refined way for all of the datasets and store these number in the `id` feature!

In [11]:
def numericalize(example, vocab):
    id = vocab.lookup_indices(example["tokens"])
    return {"id": id}

In [12]:
train_dataset = train_dataset.map(numericalize, fn_kwargs={"vocab": vocab})
validation_dataset = validation_dataset.map(numericalize, fn_kwargs={"vocab": vocab})
test_dataset = test_dataset.map(numericalize, fn_kwargs={"vocab": vocab})

Converting to torch tensors to be loaded into Transformer!

In [13]:
train_dataset = train_dataset.with_format(type="torch", columns=["id", "label"])
validation_dataset = validation_dataset.with_format(type="torch", columns=["id", "label"])
test_dataset = test_dataset.with_format(type="torch", columns=["id", "label"])

## Data Loader

Let's now pad the sequences to make them of equal lengths and then load the dataset into pytorch data loaders.

In [14]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_ids = [i["id"] for i in batch]
        batch_ids = torch.nn.utils.rnn.pad_sequence(
            batch_ids, padding_value=pad_index, batch_first=True
        )
        batch_label = [i["label"] for i in batch]
        batch_label = torch.stack(batch_label)
        batch = {"id": batch_ids, "label": batch_label}
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [15]:
# Batch Size
batch_size = 256

train_data_loader = get_data_loader(train_dataset, batch_size, pad_index, shuffle=True)
validation_data_loader = get_data_loader(validation_dataset, batch_size, pad_index)
test_data_loader = get_data_loader(test_dataset, batch_size, pad_index)

You can think of pytorch data loaders as an iterable over which you can loop and get batches one by one!

In [16]:
for batch in train_data_loader:
    print(batch["id"].shape)

torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size(

# Transformers Neural Network

In [24]:
ex_review = train_dataset[-3]["id"][:5]
token_embedding = torch.nn.Embedding(embedding_dim=7, num_embeddings=len(vocab))
emb_token = token_embedding(ex_review)
emb_token

tensor([[ 1.7333, -0.2087, -0.1684,  0.8662, -0.7795,  1.2341, -0.6230],
        [-1.3696, -0.8951,  0.8114, -1.0988, -1.2615,  0.4062, -0.3548],
        [-0.4408,  0.2652, -1.0598,  1.9366,  0.5431, -1.2966, -0.1174],
        [-0.8789, -0.2537, -0.0356,  0.8743,  0.3683,  1.5304,  1.6146],
        [ 0.9612,  0.1940, -0.7919, -1.4849, -0.6838,  0.1440,  1.0798]],
       grad_fn=<EmbeddingBackward0>)

In [27]:
np.save("../anim/ex_review_emb.npy", emb_token.detach().numpy())

In [29]:
emb_token_ = np.load("../anim/ex_review_emb.npy")
emb_token_.shape

(5, 7)

In [31]:
np.round(emb_token_, 2)

array([[ 1.73, -0.21, -0.17,  0.87, -0.78,  1.23, -0.62],
       [-1.37, -0.9 ,  0.81, -1.1 , -1.26,  0.41, -0.35],
       [-0.44,  0.27, -1.06,  1.94,  0.54, -1.3 , -0.12],
       [-0.88, -0.25, -0.04,  0.87,  0.37,  1.53,  1.61],
       [ 0.96,  0.19, -0.79, -1.48, -0.68,  0.14,  1.08]], dtype=float32)